In [2]:
import pandas as pd
import seaborn as sns
from sqlalchemy import create_engine
engine = create_engine("mysql+pymysql://{user}:{pw}@{host}:{port}/{db}?charset=utf8mb4"
                       .format(user="openalex_ym",
                               pw="openalex.ym",
                               host="172.18.18.167",
                               port=3306,
                               db="openalex2022"),
                       )

#### Import AFT Data set

##### mentorship

In [2]:
Mtoship = pd.read_csv('cellbio.csv')

In [3]:
Mtoship.head(5)

,Unnamed: 0,CID,MenteeID,MentorID,MentorshipType,Institution,InstitutionMAGID,StartYear,StopYear
0,321,362,329,327,2,"University of California, Berkeley",95457486.0,1988,1992
1,1271,1438,412,1835,1,California Institute of Technology,122411786.0,-1,1987
2,1381,1551,1273,359,2,Yale University,32971472.0,-1,1984
3,1874,2118,1696,751,1,Johns Hopkins University,145311948.0,-1,1997
4,1888,2132,359,527,0,Johns Hopkins University,145311948.0,-1,-1


In [4]:
MentorID = list(set(Mtoship.MentorID))
print(len(MentorID))

7750


In [5]:
pid_mag_openalex = pd.read_csv('pid_mag_openalex.csv')

In [6]:
pid_mag_openalex.head(5)

,PID,MAGAuthorID,author_id
0,1,2.171828e+09,A2171827615
1,2,2.082173e+09,A2082172561
2,3,2.110166e+09,A2110165986
3,4,2.183264e+09,A2183263527
4,5,2.156587e+09,A2156586888


In [7]:
mentor = pid_mag_openalex['author_id'][(pid_mag_openalex.PID.isin(MentorID))&(~pid_mag_openalex.author_id.isnull())]
print(len(mentor))

6949


In [8]:
mentor.to_csv('target_author.csv',index=0)

#### find authorship of target_author

In [9]:
target_author = mentor

In [11]:
mentee_groups = []
interval = 5000
start_point = 0
while True:
    end_point = start_point + interval
    try:
        temp = target_author[start_point:end_point]
        mentee_groups.append(temp)
    except:
        temp = target_author[start_point:]
        mentee_groups.append(temp)
        
    start_point = end_point
    if end_point>len(target_author):
        break
print(len(mentee_groups))

2


In [12]:
l = len(mentee_groups)
print(l)
for i in range(l):
    temp =  mentee_groups[i]
    sql_query = """select works_authorships.work_id, works_authorships.author_id

                    from works_authorships 

                    where works_authorships.author_id in {}
                    """.format(tuple(temp))


    data = pd.read_sql(sql_query, con=engine)
    # merge data
    if i==0:
        tp = data
    else:
        tp = pd.concat((tp,data))
    if i%10==0:
        print(i)

Data = tp

2
0


In [13]:
Data.head(5)

,work_id,author_id
0,W1985765285,A100549476
1,W2405137022,A100549476
2,W2889237557,A100549476
3,W2998601278,A100549476
4,W1964706578,A100549476


In [14]:
target_work = Data.work_id.drop_duplicates().tolist()
print(len(target_work))

903183


##### find full authorship

In [15]:
paper_groups = []
interval = 5000
start_point = 0
while True:
    end_point = start_point + interval
    try:
        temp = target_work[start_point:end_point]
        paper_groups.append(temp)
    except:
        temp = target_work[start_point:]
        paper_groups.append(temp)
        
    start_point = end_point
    if end_point>len(target_work):
        break
len(paper_groups)

181

In [16]:
l = len(paper_groups)
n0 = 0
n1 = l
for i in range(n0,n1):
    temp =  paper_groups[i]
    sql_query = """select *

                    from works_authorships 

                    where works_authorships.work_id in {}
                    """.format(tuple(temp))


    data = pd.read_sql(sql_query, con=engine)
    # merge data
    if i==n0:
        tp = data
    else:
        tp = pd.concat((tp,data))
    # print(i)
    
    if i%10==0:
        print(i)

Data = tp

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180


In [17]:
inst_ids = list(Data.institution_id)
inst_ids1 = [k.replace('\r', '') for k in inst_ids]
Data['institution_id'] = inst_ids1

In [18]:
Data.sample(5)

,work_id,author_position,author_id,institution_id
25085,W2621065569,middle,A2181956068,
27903,W3199289237,middle,A1997613316,I4210140103
4216,W1993482400,first,A2181472056,I20089843
29303,W2934954259,middle,A2935327374,I130769515
8062,W2043646309,middle,A2125259716,I157725225


In [19]:
Data.to_csv('G1_mentee_authorship.csv',index=0)

In [20]:
len(set(target_author))

6830

In [21]:
all_author = set(Data['author_id'])
set(target_author).intersection(all_author) == set(target_author)

True

In [22]:
len(set(target_author).intersection(all_author))

6830

##### find career info of all authors

In [23]:
all_mentee = Data.author_id.drop_duplicates().tolist()

In [24]:
mentee_groups = []
interval = 5000
start_point = 0
while True:
    end_point = start_point + interval
    try:
        temp = all_mentee[start_point:end_point]
        mentee_groups.append(temp)
    except:
        temp = all_mentee[start_point:]
        mentee_groups.append(temp)
        
    start_point = end_point
    if end_point>len(all_mentee):
        break
len(mentee_groups)

286

In [25]:
l = len(mentee_groups)
print(l)
n0 = 0
n1 = l
for i in range(n0,n1):
    temp =  mentee_groups[i]
    sql_query = """select works_authorships.work_id, works_authorships.author_id

                    from works_authorships 

                    where works_authorships.author_id in {}
                    """.format(tuple(temp))


    data = pd.read_sql(sql_query, con=engine)
    # merge data
    if i==0:
        tp = data
    else:
        tp = pd.concat((tp,data))
    if i%10==0:
        print(i)

Data = tp

286
0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280


In [ ]:
Data.to_csv('G1_mentee_allWorks.csv',index=0)

##### find publication_year

In [3]:
Data = pd.read_csv('G1_mentee_allWorks.csv')

In [4]:
paper = list(set(Data.work_id))
print(len(paper))

paper_groups = []
interval = 5000
start_point = 0
while True:
    end_point = start_point + interval
    try:
        temp = paper[start_point:end_point]
        paper_groups.append(temp)
    except:
        temp = paper[start_point:]
        paper_groups.append(temp)
        
    start_point = end_point
    if end_point>len(paper):
        break
print(len(paper_groups))

23647108
4730


In [5]:
l = len(paper_groups)
print(l)
for i in range(l):
    temp =  paper_groups[i]
    sql_query = """select works.id, works.publication_year

                    from works 

                    where works.id in {}
                    """.format(tuple(temp))


    data = pd.read_sql(sql_query, con=engine)
    # merge data
    if i==0:
        tp = data
    else:
        tp = pd.concat((tp,data))
    if i%10==0:
        print(i)

4730
0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930


KeyboardInterrupt: 

In [ ]:
tp = tp.rename(columns={'id':'work_id'})
tp.to_csv('G1_allWorks_year.csv',index=0)

In [ ]:
Data = Data.merge(tp,on='work_id',how='left')
min_year = Data[['author_id','publication_year']].drop_duplicates()
first_pub_year = min_year.groupby('author_id')['publication_year'].min().reset_index(name='first_pub_year')
last_pub_year = min_year.groupby('author_id')['publication_year'].max().reset_index(name='last_pub_year')

career_span = first_pub_year.merge(last_pub_year,on='author_id',how='left')
career_span['career_len'] = career_span['last_pub_year'] - career_span['first_pub_year']

In [ ]:
career_span.to_csv('G1_mentee_careerSpan.csv',index=0)

##### Tabel for GroupSize(G2)

In [ ]:
authorship = pd.read_csv('G1_mentee_authorship.csv')
authorship = authorship.merge(career_span,on='author_id',how='left')
work_year = pd.read_csv('G1_allWorks_year.csv')
authorship = authorship.merge(work_year,on='work_id',how='left')
authorship['scientific_age'] = authorship['publication_year'] - authorship['first_pub_year']

In [ ]:
authorship.to_csv('G1_mentee_authorship.csv',index=0)

In [ ]:
def inst_completeness(tp_df):
    not_null_num = len(tp_df[~tp_df.institution_id.isnull()])
    ratio = not_null_num/len(tp_df)
    
    return ratio

In [ ]:
data = authorship.groupby('work_id').apply(inst_completeness).reset_index(name = 'ratio_inst.')
data.to_csv('G1_mentee_inst_completeness.csv',index=0)